# Revista Contemporânea de Contabilidade – RCC

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
url_search_text = 'https://revistas.ufpr.br/rcc/search/search?query=__query__&searchJournal=49&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=01&dateFromDay=1&dateFromYear=2012&dateToMonth=12&dateToDay=31&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'
links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link

'https://revistas.ufpr.br/rcc/article/view/52350'

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [8]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [9]:
captura_titulo(soup)['Título']

'Revisão Bibliográfica do Perfil Temático e Metodológico da Pesquisa em Disclosure Voluntário veiculada em Edições Eletrônicas de Periódicos Brasileiros e Internacionais entre 2002 e 2018.'

In [10]:
def captura_data(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [11]:
captura_data(soup)

{'Data de Publicação': '2019'}

In [12]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [13]:
captura_resumo(soup)

{'Resumo': 'Levantamento efetuado na base Scopus demonstrou que o tema da divulgação voluntária passou por gradativa ascensão entre as pesquisas acadêmicas da área de Negócios, Gestão e Contabilidade, passando de 67 artigos no período 1979-1999 a 887 artigos no período 2000-2018. A relevância do tema foi acentuada com as publicações de Verrecchia (2001) e Dye (2001). Em vista deste cenário, questionou-se: Qual é o perfil temático e metodológico das investigações científicas em “Divulgação Voluntária” publicadas entre 2002 e 2018 nas versões eletrônicas de periódicos da área de Contabilidade? Para tanto, empregou-se técnicas de bibliometria e análise de conteúdo. As amostras selecionadas abrangeram 247 artigos internacionais e 71 artigos nacionais. Como principais resultados, os assuntos mais abordados dentro da temática foram Disclosure Ambiental, Responsabilidade Social Corporativa, Informações Socioambientais e de Sustentabilidade, Itens específicos dos demonstrativos e NEs, Eficiênc

In [14]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [15]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Análises Bibliométricas. Divulgação Voluntária. Índices Cienciométricos.'}

In [16]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find('div', {'id': 'authorString'})
        authors_full = authors_tag.find('em')
        autores = authors_full.text.split(', ')
    except:
        pass

    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [17]:
authors_tag = soup.find('div', {'id': 'authorString'})
authors_full = authors_tag.find('em')
autores = authors_full.text.split(', ')

autores

['Aline Andrade Barbosa da Silva',
 'Renata Orsato',
 'Luciano Márcio Scherer',
 'Márcia Maria dos Santos Bortolocci Espejo']

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Aline Andrade Barbosa da Silva',
 'Autor 2': 'Renata Orsato',
 'Autor 3': 'Luciano Márcio Scherer',
 'Autor 4': 'Márcia Maria dos Santos Bortolocci Espejo',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [20]:
captura_afiliacao(soup)

{'Afilição 1': '',
 'Afilição 2': '',
 'Afilição 3': '',
 'Afilição 4': '',
 'Afilição 5': '',
 'Afilição 6': ''}

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Contabilidade e Controladoria – RC&C'}
    url_search_text = 'https://revistas.ufpr.br/rcc/search/search?query=__query__&searchJournal=49&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=01&dateFromDay=1&dateFromYear=2012&dateToMonth=12&dateToDay=31&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'

    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if 'Pré-textual' in captura_titulo(soup)['Título'] or 'CAP Accounting and Management' in captura_titulo(soup)['Título']:
            continue
            
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2019-01-01,Revisão Bibliográfica do Perfil Temático e Met...,Levantamento efetuado na base Scopus demonstro...,Análises Bibliométricas. Divulgação Voluntária...,4,Aline Andrade Barbosa da Silva,Renata Orsato,Luciano Márcio Scherer,Márcia Maria dos Santos Bortolocci Espejo,,,,,,,,,https://revistas.ufpr.br/rcc/article/view/52350,Revista de Contabilidade e Controladoria – RC&C
1,2019-01-01,DA BARRAGEM DE FUNDÃO À CONTABILIDADE: UMA ANÁ...,"Em novembro de 2015 no município de Mariana, M...",Avaliação de Desempenho. Contabilidade Ambient...,1,Herberty Cerqueira Lima,,,,,,,,,,,,https://revistas.ufpr.br/rcc/article/view/71041,Revista de Contabilidade e Controladoria – RC&C
2,2013-01-01,UMA AVALIAÇÃO DOS EFEITOS DOS SISTEMAS DE INCE...,As organizações estão inseridas em um mercado ...,Desempenho econômico. Sistemas de incentivos. ...,5,Guillermina Tannuri,Sueli Farias,Ernesto Fernando Rodrigues Vicente,Hans Michael van Bellen,Luiz Alberton,,,,,,,,https://revistas.ufpr.br/rcc/article/view/27292,Revista de Contabilidade e Controladoria – RC&C


In [23]:
df.to_csv('../data/RCC.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2019-01-01,Revisão Bibliográfica do Perfil Temático e Met...,Levantamento efetuado na base Scopus demonstro...,Análises Bibliométricas. Divulgação Voluntária...,4,Aline Andrade Barbosa da Silva,Renata Orsato,Luciano Márcio Scherer,Márcia Maria dos Santos Bortolocci Espejo,,,,,,,,,https://revistas.ufpr.br/rcc/article/view/52350,Revista de Contabilidade e Controladoria – RC&C
1,2019-01-01,DA BARRAGEM DE FUNDÃO À CONTABILIDADE: UMA ANÁ...,"Em novembro de 2015 no município de Mariana, M...",Avaliação de Desempenho. Contabilidade Ambient...,1,Herberty Cerqueira Lima,,,,,,,,,,,,https://revistas.ufpr.br/rcc/article/view/71041,Revista de Contabilidade e Controladoria – RC&C
2,2013-01-01,UMA AVALIAÇÃO DOS EFEITOS DOS SISTEMAS DE INCE...,As organizações estão inseridas em um mercado ...,Desempenho econômico. Sistemas de incentivos. ...,5,Guillermina Tannuri,Sueli Farias,Ernesto Fernando Rodrigues Vicente,Hans Michael van Bellen,Luiz Alberton,,,,,,,,https://revistas.ufpr.br/rcc/article/view/27292,Revista de Contabilidade e Controladoria – RC&C
